# Esto es código auxiliar para seguir el siguiente proceso de Business Inteligence:
<img src='images/proceso.png'>

Puedes obtener <a href='https://www.kaggle.com/datasets/lokeshparab/amazon-products-dataset'>aquí</a> la fuente de datos, deberías añadirla en una carpeta data en este mismo directorio para poder extraer los datos con este notebook.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/Amazon-Products.csv", index_col=0)
df.pop('image')
df.pop('link')
df = df.set_index('name')

In [3]:
df.head()

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
name,,,,,,
"Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1 Convertible, Copper, Anti-Viral + Pm 2.5 Filter, 2023 Model, White, Gls18I3...",appliances,Air Conditioners,4.2,"2,255","₹32,999","₹58,990"
"LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.2,"2,948","₹46,490","₹75,990"
"LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Copper, Super Convertible 6-In-1 Cooling, Hd Filter With Anti Virus Protection,...",appliances,Air Conditioners,4.2,"1,206","₹34,490","₹61,990"
"LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.0,69,"₹37,990","₹68,990"
"Carrier 1.5 Ton 3 Star Inverter Split AC (Copper,ESTER Dxi, 4-in-1 Flexicool Inverter, 2022 Model,R32,White)",appliances,Air Conditioners,4.1,630,"₹34,490","₹67,790"


In [4]:
null_percentages = pd.DataFrame(100 * df.isnull().sum() / len(df)).transpose()
null_percentages

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
0,0.0,0.0,31.8707,31.8707,11.08859,3.229421


Tenemos más de un 31% de ragings a null.<br>
Consultando el kaggle original, vemos que esto indica que no ha habido ningún rating de ese producto.<br>



De aquí ya extraemos la conclusión de que en torno al 30% de los productos de amazon no llegan a tener unas ventas significativas y se quedan en un intento muy vago sin ni siquiera una opinión.

Así sabemos que no estamos perdiendo información que nos interese si eliminamos todas las filas que tengan el rating a null.<br>
Tras hacerlo, tenemos la siguiente tabla de porcentajes nulos:

In [5]:
df.dropna(inplace=True)
null_percentages = pd.DataFrame(100 * df.isnull().sum() / len(df)).transpose()
null_percentages

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
0,0.0,0.0,0.0,0.0,0.0,0.0


Ahora damos el tipo correspondiente a cada columna

In [6]:
# Dar formato de número a los ratings y no_of_ratings
for i in ['no_of_ratings', 'discount_price', 'actual_price']:
    df[i] = df[i].apply(lambda x: x.replace(',', ''))
    
df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce')
df.dropna(subset=['no_of_ratings'], inplace=True)
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df.dropna(subset=['ratings'], inplace=True)
    


# # Convertimos de rupias a dólares
# # A veces se cae el servidor; abajo dejo la conversión en el momento de la realización de este notebook
# from forex_python.converter import CurrencyRates

# c = CurrencyRates()
# tasa_conversion = c.get_rate('INR', 'USD')

tasa_conversion = 0.121772

df['discount_price'] = df['discount_price'].apply(lambda x: x[1:]) # quitamos el símbolo de rupias
df['discount_price'] = pd.to_numeric(df['discount_price'], errors='coerce')
df.dropna(subset=['discount_price'], inplace=True)
df['discount_price'] = df['discount_price'] * tasa_conversion

df['actual_price'] = df['actual_price'].apply(lambda x: x[1:]) # quitamos el símbolo de rupias
df['actual_price'] = pd.to_numeric(df['actual_price'], errors='coerce')
df.dropna(subset=['actual_price'], inplace=True)
df['actual_price'] = df['actual_price'] * tasa_conversion



df.describe()

,ratings,no_of_ratings,discount_price,actual_price
count,334963.000000,334963.000000,334963.000000,3.349630e+05
mean,3.814368,853.864540,214.198090,4.017081e+03
std,0.754169,8890.977903,778.668253,2.083015e+06
min,1.000000,1.000000,0.974176,1.217720e+00
25%,3.500000,4.000000,43.716148,1.095948e+02
50%,3.900000,20.000000,72.941428,1.825362e+02
75%,4.200000,131.000000,146.004628,3.408398e+02
max,5.000000,589547.000000,152213.782280,1.205543e+09


Tras borrar todas las columnas nos quedamos con algo más de 100k; para un fin educativo es suficiente

In [7]:
df.head()

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
name,,,,,,
"Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1 Convertible, Copper, Anti-Viral + Pm 2.5 Filter, 2023 Model, White, Gls18I3...",appliances,Air Conditioners,4.2,2255.0,4018.354228,7183.33028
"LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.2,2948.0,5661.180280,9253.45428
"LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Copper, Super Convertible 6-In-1 Cooling, Hd Filter With Anti Virus Protection,...",appliances,Air Conditioners,4.2,1206.0,4199.916280,7548.64628
"LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.0,69.0,4626.118280,8401.05028
"Carrier 1.5 Ton 3 Star Inverter Split AC (Copper,ESTER Dxi, 4-in-1 Flexicool Inverter, 2022 Model,R32,White)",appliances,Air Conditioners,4.1,630.0,4199.916280,8254.92388


Procedemos a pasar las tablas a una base de datos SQLite con la siguiente estructura en estrella:

<img src="images/tablas_hechos.png">

Lo primero es crear las tablas:

In [8]:
categorias = df.copy()
categorias.set_index('sub_category', inplace=True)
categorias.pop('ratings')
categorias.pop('no_of_ratings')
categorias.pop('discount_price')
categorias.pop('actual_price')
categorias = categorias.groupby('sub_category').aggregate({'main_category': 'first'})
categorias

,main_category
sub_category,
Air Conditioners,appliances
All Appliances,appliances
All Car & Motorbike Products,car & motorbike
All Electronics,"tv, audio & cameras"
All Exercise & Fitness,sports & fitness
...,...
Washing Machines,appliances
Watches,accessories
Western Wear,women's clothing


In [9]:
ventas = df.copy()
ventas.pop('main_category')
ventas

,sub_category,ratings,no_of_ratings,discount_price,actual_price
name,,,,,
"Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1 Convertible, Copper, Anti-Viral + Pm 2.5 Filter, 2023 Model, White, Gls18I3...",Air Conditioners,4.2,2255.0,4018.354228,7183.330280
"LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",Air Conditioners,4.2,2948.0,5661.180280,9253.454280
"LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Copper, Super Convertible 6-In-1 Cooling, Hd Filter With Anti Virus Protection,...",Air Conditioners,4.2,1206.0,4199.916280,7548.646280
"LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",Air Conditioners,4.0,69.0,4626.118280,8401.050280
"Carrier 1.5 Ton 3 Star Inverter Split AC (Copper,ESTER Dxi, 4-in-1 Flexicool Inverter, 2022 Model,R32,White)",Air Conditioners,4.1,630.0,4199.916280,8254.923880
...,...,...,...,...,...
BE SAVAGE Pink I Purple You Hoodie Suitable for Summer & Winter for Men,Yoga,4.6,6.0,86.579892,121.650228
Adidas Regular Fit Men's Track Tops,Yoga,3.2,9.0,419.991628,560.029428
Redwolf Noice Toit Smort - Hoodie (Black),Yoga,2.0,2.0,146.004628,243.422228


# Ya tenemos los datos bien estructurados, solamente nos queda añadirlos a una base de datos:

In [10]:
import sqlite3

# Creamos una conexión a la base de datos SQLite
conn = sqlite3.connect('hechos.db')

# Pasamos los DataFrames a la base de datos
categorias.to_sql('categorias', conn, if_exists='replace', index=True)
ventas.to_sql('ventas', conn, if_exists='replace', index=True, 
           dtype={'name': 'INT', 'sub_category': 'TEXT',
                  'FOREIGN KEY(subcategory)': 'REFERENCES categorias(subcategorias)',
                  'ratings': 'FLOAT', 'no_of_ratings': 'FLOAT',
                  'discount_price': 'FLOAT', 'actual_price': 'FLOAT'})

# Cerramos la conexión a la base de datos
conn.close()

También podemos crear archivos csv para estas dos tablas por si las necesitamos en algún momento

In [11]:
categorias.to_csv('categorias.csv')
df.to_csv('ventas.csv')

In [12]:

df

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
name,,,,,,
"Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1 Convertible, Copper, Anti-Viral + Pm 2.5 Filter, 2023 Model, White, Gls18I3...",appliances,Air Conditioners,4.2,2255.0,4018.354228,7183.330280
"LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.2,2948.0,5661.180280,9253.454280
"LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Copper, Super Convertible 6-In-1 Cooling, Hd Filter With Anti Virus Protection,...",appliances,Air Conditioners,4.2,1206.0,4199.916280,7548.646280
"LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.0,69.0,4626.118280,8401.050280
"Carrier 1.5 Ton 3 Star Inverter Split AC (Copper,ESTER Dxi, 4-in-1 Flexicool Inverter, 2022 Model,R32,White)",appliances,Air Conditioners,4.1,630.0,4199.916280,8254.923880
...,...,...,...,...,...,...
BE SAVAGE Pink I Purple You Hoodie Suitable for Summer & Winter for Men,sports & fitness,Yoga,4.6,6.0,86.579892,121.650228
Adidas Regular Fit Men's Track Tops,sports & fitness,Yoga,3.2,9.0,419.991628,560.029428
Redwolf Noice Toit Smort - Hoodie (Black),sports & fitness,Yoga,2.0,2.0,146.004628,243.422228
